In [1]:
from konlpy.tag import Kkma
from konlpy.tag import Komoran
from konlpy.tag import Hannanum
import MeCab
import nltk

#윈도우 mecab 불러오기

from konlpy.tag import Okt
import re

#형태소 분석을 위한 객체 생성
kkma=Kkma()
komoran=Komoran()
hannanum=Hannanum()
#윈도우에서 Mecab생성
mecab=MeCab.Tagger()
okt=Okt()

In [2]:
stop_words=[
    '이','있','하','것','들','그','되','이','보','않','없','나','같','없','가','때','들','주','등','같',
    '년','한','지','오','말','일','나오','그렇','위하','때문','그것','두','알','그러나','받','못하','요즘',
    '그런','또','더','그리고','좋','중','따르','가지','씨','시키','살','될','안','데','내','경우','다시','그녀',
    '이런','앞','나','된','잘','놓','위','위해','했','받','해','대해','해서','통해','후기','사','점','끝',
    '이번','큰','와서','동안','지금','오늘','코로나','네이버','위한','넣','크','맞','날','할','속','남','그때',
    '한다','작','적','많','왔','해도','된다','전','이르','후'
]
print(len(stop_words))

100


In [3]:
#여가 딕셔너리 검색어 생성(검색/출력용)
leisure_dict_search=[
    #문화예술관람활동
    ['미술_전시회_관람','사진_전시회_관람','건축_전시회_관람','디자인_전시회_관람',
    '박물관_관람',
    '클래식_음악_연주회_관람','오페라_음악_연주회_관람',
    '국악_전통공연_관람','민속놀이_공연_관람',
    '연극공연_관람','뮤지컬_관람',
    '무용_공연_관람',
    '영화_관람',
    '쇼_공연_관람','콘서트_공연_관람','마술_쇼_공연_관람'],
    #문화예술참여활동
    ['문학_행사_참여',
    '글짓기_참여' ,'독서_토론_참여',
    '그림_활동_참여','서예_활동_참여','조각_활동_참여','디자인_활동_참여','도예_활동_참여','만화_활동_참여',
    '악기_연주_참여','노래교실 참여',
    '춤_참여','무용_참여','발레_참여','한국무용_참여','현대무용_참여','방송댄스_참여','스트릿댄스_참여','비보잉_참여',
    '사진_촬영',
    '사물놀이_참여','줄타기_참여'],
    #스포츠관람활동
    ['축구_경기장_방문_관람' ,'야구_경기장_방문_관람','농구_경기장_방문_관람','배구_경기장_방문_관람',
    '축구_경기_TV_관람','야구_경기_TV_관람','농구_경기_TV_관람','배구_경기_TV_관람',
    '태권도_경기_관람','유도_경기_관람','합기도_경기_관람','검도_경기_관람','권투_경기_관람',
    'e-스포츠_경기_관람'],
    #스포츠참여활동
    ['농구_참여','배구_참여','야구_참여','축구_참여','족구_참여',
    '테니스_참여','스쿼시_참여',
    '당구_참여','포켓볼_참여',
    '볼링_참여','탁구_참여',
    '골프_참여',
    '수영_참여' ,
    '윈드서핑_참여','수상스키_참여','스킨스쿠버다이빙_참여','래프팅_참여','요트_참여',
    '스노보드_참여','스키_참여',
    '아이스스케이트_참여','아이스하키_참여',
    '헬스_참여','보디빌딩_참여','에어로빅_참여',
    '요가_참여','필라테스_참여','태보_운동_참여',
    '배드민턴_참여','줄넘기_참여','맨손_체조_참여','스트레칭_참여','훌라후프_참여',
    '육상_참여','조깅_참여','빠르게_걷기_참여',
    '태권도_참여','유도_참여','합기도_참여','검도_참여','권투_참여',
    '탱고_참여','왈츠_참여','자이브_참여','맘보_참여','폴카_참여','차차차_참여',
    '사이클링_참여','자전거_참여',
    '인라인_스케이트_참여',
    '승마_참여','암벽등반_참여','철인삼종경기_참여','서바이벌_참여'],
    #관광활동
    ['문화유적_방문',
    '자연명승_방문','풍경_관람',
    '삼림욕',
    '국내캠핑',
    '해외여행',
    '소풍','야유회',
    '온천','해수욕',
    '유람선_타기',
    '테마파크_가기','놀이공원_가기','동물원_가기','식물원_가기',
    '지역축제_참가',
    '자동차_드라이브'],
    #취미오락활동
    ['콜렉션','스크랩북',
    '십자수','비즈공예', 'DIY','꽃꽂이',
    '요리하기','다도',
    '반려동물_키우기',
    '노래방_가기',
    '집_인테리어','자동차_인테리어',
    '등산',
    '낚시',
    '홈페이지_관리','블로그_관리',
    '인터넷_검색','1인_미디어_제작','SNS',
    '온라인_게임','모바일_게임','콘솔_게임',
    '보드게임','퍼즐','큐브_맞추기',
    '바둑','장기','체스',
    '경마','경륜','카드놀이','고스톱','마작','복권구입',
    '쇼핑','외식',
    '음주',
    '독서','웹소설',
    '만화보기','웹툰보기',
    '피부관리','헤어관리','네일아트','마사지','성형',
    '어학_공부','기술_공부','자격증_공부',
    '방탈출_카페_체험','VR_카페_체험','낚시카페_체험',
    '원예','화단_가꾸기'],
    #휴식활동
    ['산책','걷기',
    '목욕','사우나','찜질방',
    '낮잠',
    'TV_시청',
    '모바일_컨텐츠','동영상_시청',
    '라디오_청취','팟캐스트_청취',
    '음악_감상',
    '신문_보기','잡지_보기',
    '아무것도_안_하기'],
    #사회 및 기타활동
    ['사회봉사',
    '종교활동',
    '클럽_가기','나이트_가기','디스코_가기','캬바레_가기',
    '가족_방문','친지_방문',
    '잡담','통화하기','문자보내기',
    '계모임','동창회','사교파티',
    '친구_만남','이성교제','미팅','소개팅',
    '동호회_모임']
]

In [4]:
online_keywords=['온라인', '1인', '혼자', '집', '통화', '유튜브', '방송', '만들',
                '비대면', 'SNS', '휴대폰', '블로그', '잠', 'TV', '관람', '게임',
                '웹', '사이트', '간단', '홈', '배워', '중계', '시청','비대', '전화',
                 '문자', '발송', '인터넷', '청취','셀프']
offline_keywords=['오프라인', '공간', '곳', '길', '입장', '방문', '룸', '대면',
                 '분위기', '여행', '관광', '산책', '지역', '시설', '출입', '모임',
                 '이용', '체육관', '공원', '풍경', '가족', '스포츠', '산']

In [5]:
def mecab_extract(text):
    nouns = []
    temp=[]
    #obj_list=obj.split("_")
    # 우리가 원하는 TOKEN\tPOS의 형태를 추출하는 정규표현식.
    pattern = re.compile(".*\t[A-Z]+") 
    
    # 패턴에 맞는 문자열을 추출하여 konlpy의 mecab 결과와 같아지도록 수정.
    for token in mecab.parse(text).splitlines()[:-1]:
        temptuple=pattern.match(token)
        if(temptuple!=None):#몇 구문에서 공백도 처리하는 문제 발생. 파이썬스러운 구문으로 하기에는 문제가 좀 있음
            temp.append(tuple(temptuple.group(0).split("\t")))
    
    # 추출한 token중에 일반명사, 고유명사, 동사, 형용사, 어근 추출
    for token in temp:
        if token[0] in stop_words: #불용어 100개 제거
            continue
        if token[1] == "NNG" or token[1] == "NNP" or token[1] == "VV" or token[1] == "VA" or token[1] == "XR":
            nouns.append(token[0])
        
    return nouns

def mecab_morphs(text):
    morphs = []
    
    # 우리가 원하는 TOKEN\tPOS의 형태를 추출하는 정규표현식.
    pattern = re.compile(".*\t[A-Z]+") 
    
    # 패턴에 맞는 문자열을 추출하여 konlpy의 mecab 결과와 같아지도록 수정.
    temp = [tuple(pattern.match(token).group(0).split("\t")) for token in mecab.parse(text).splitlines()[:-1]]
        
    # 추출한 token중에 문자열만 선택.
    for token in temp:
        morphs.append(token[0])
    
    return morphs

def mecab_pos(text):
    pos = []
    # 우리가 원하는 TOKEN\tPOS의 형태를 추출하는 정규표현식.
    pattern = re.compile(".*\t[A-Z]+") 
    
    # 패턴에 맞는 문자열을 추출하여 konlpy의 mecab 결과와 같아지도록 수정.
    pos = [tuple(pattern.match(token).group(0).split("\t")) for token in mecab.parse(text).splitlines()[:-1]]
        
    return pos

In [11]:
description=[]
for obj in leisure_dict_search[6]:
    fread=open("./Leisures/Text/"+obj+".txt", 'r',encoding='UTF8')
    templist=fread.read()
    tempresult=mecab_extract(templist)
    text=nltk.Text(tempresult,name=obj)
    description.append(text.vocab())
print(len(description))

15


In [21]:
description[0]

[('산책', 4874),
 ('공원', 2470),
 ('길', 2222),
 ('곳', 1577),
 ('시간', 1147),
 ('걷', 1041),
 ('꽃', 982),
 ('사람', 980),
 ('먹', 973),
 ('사진', 954),
 ('집', 912),
 ('강아지', 835),
 ('아이', 692),
 ('나무', 690),
 ('생각', 667),
 ('여행', 666),
 ('코스', 665),
 ('찍', 646),
 ('모습', 639),
 ('보이', 637),
 ('숲', 630),
 ('풍경', 607),
 ('서울', 548),
 ('여름', 545),
 ('공간', 537),
 ('산책로', 537),
 ('운동', 534),
 ('볼', 533),
 ('마음', 529),
 ('자연', 524),
 ('물', 515),
 ('따라', 513),
 ('시작', 506),
 ('정도', 502),
 ('걸', 492),
 ('호수', 481),
 ('카페', 469),
 ('찾', 466),
 ('시원', 464),
 ('날씨', 458),
 ('주변', 444),
 ('피', 440),
 ('아침', 439),
 ('비', 432),
 ('마을', 428),
 ('거리', 419),
 ('하늘', 407),
 ('다양', 400),
 ('이용', 393),
 ('기분', 386),
 ('만들', 381),
 ('즐기', 375),
 ('옆', 373),
 ('앉', 363),
 ('방문', 358),
 ('힐링', 355),
 ('타', 348),
 ('저녁', 347),
 ('쉬', 347),
 ('주차장', 344),
 ('위치', 343),
 ('가족', 343),
 ('반려', 338),
 ('산', 334),
 ('바람', 332),
 ('동네', 329),
 ('밤', 326),
 ('느낌', 324),
 ('책', 324),
 ('추천', 322),
 ('눈', 321),
 ('아름다운', 312),
 ('

In [22]:
if '공원' in description[0]:
    print("true")

- 온/오프라인 분석 해야됨

- 온라인 키워드(비대면 키워드)
  - 온라인, 1인, 혼자, 집, 통화, 유튜브, 방송, 만들, 비대면, SNS, 휴대폰, 블로그, 잠, TV, 관람, 게임, 웹, 사이트, 간단, 홈, 배워, 중계, 시청, 비대, 전화, 문자, 발송, 인터넷, 청취, 셀프
- 오프라인 키워드(대면 키워드)
  - 오프라인, 공간, 곳, 길, 입장, 방문, 룸, 대면, 분위기, 여행, 관광, 산책, 지역, 시설, 출입, 모임, 이용, 체육관, 공원, 풍경, 가족, 스포츠, 산, 가기, 외식, 전시, 활동, 만남, 시민, 팀, 센터

- 아래는 온/오프라인 판별
  - 오프라인 : 온라인 = 3 : 7일 경우 온라인으로 판별(오프라인/온라인<=0.43 라면 온라인)
  - 오프라인 : 온라인 = 7 : 3일 경우 오프라인으로 판별(오프라인/온라인>=2.33 라면 오프라인)
  - 그 외에는 하이브리드로 판별(둘 다 가능)

In [132]:
fread=open("./Leisures/Text/.txt", 'r',encoding='UTF8')
templist=fread.read()
tempresult=mecab_nouns(templist)
text=nltk.Text(tempresult,name='아무것도_안_하기')
esports=text.vocab().most_common(500)

In [79]:
type(esports)

list

In [86]:
offlinecnt=0
onlinecnt=0
for obj in esports:
    if obj[0] in offline_keywords:
        offlinecnt+=obj[1]
    elif obj[0] in online_keywords:
        onlinecnt+=obj[1]
print(offlinecnt/onlinecnt)

1.4350736278447123


In [39]:
#mecab일일이 하면 어려우니 pickle을 이용한 각 500개 리스트 생성
import pickle
for row in leisure_dict_search:
    for obj in row:
        if(obj=='철인삼종경기_참여'):
            continue
        fread=open("./Leisures/Text/"+obj+".txt", 'r',encoding='UTF8')
        templist=fread.read()
        tempresult=mecab_extract(templist)
        text=nltk.Text(tempresult,name=obj)
        commons=text.vocab().most_common(500)
        f = open("./Leisures/Description_Data/"+obj+".pkl", 'wb')
        pickle.dump(commons,f)
        f.close()

In [ ]:
#온/오프라인 검색
offlinecnt=0
onlinecnt=0
for obj in esports:
    if obj[0] in offline_keywords:
        offlinecnt+=obj[1]
    elif obj[0] in online_keywords:
        onlinecnt+=obj[1]
print(offlinecnt/onlinecnt)

In [29]:
import pickle

In [31]:
f = open("./Leisures/Description_Data/철인삼종경기_참여.pkl", 'wb')
pickle.dump(esports,f)

In [38]:
fread=open("./Leisures/Description_Data/철인삼종경기_참여.pkl", 'rb')
templist=pickle.load(fread)
type(templist)

<class 'str'>


In [28]:
new_list=str(esports)
print(new_list)

[('之', 19273), ('사람', 14101), ('선생', 5134), ('신', 5061), ('죽', 5021), ('먹', 4994), ('마음', 4333), ('생각', 4155), ('爲', 3689), ('先生', 3431), ('나라', 3181), ('물', 3122), ('색소', 3071), ('뒤', 2745), ('힘', 2721), ('세계', 2640), ('인간', 2604), ('병', 2499), ('可', 2479), ('臣', 2244), ('시간', 2213), ('우주', 2186), ('金', 2179), ('이상', 2176), ('상', 2176), ('자신', 2166), ('돼', 2160), ('그러', 2137), ('따라', 2132), ('시', 2125), ('길', 2117), ('神', 2106), ('세상', 2063), ('전하', 1980), ('집', 1948), ('불', 1938), ('암', 1932), ('법', 1931), ('산', 1881), ('쓰', 1864), ('대한', 1854), ('피', 1851), ('뜻', 1845), ('죽염', 1835), ('몸', 1832), ('사진', 1831), ('시작', 1825), ('서울', 1821), ('경기', 1798), ('공', 1792), ('곳', 1780), ('鄭', 1745), ('정신', 1700), ('타', 1695), ('時', 1692), ('화', 1658), ('아들', 1637), ('다음', 1609), ('하루', 1605), ('자연', 1603), ('정도', 1602), ('분자', 1600), ('모르', 1543), ('간', 1537), ('생기', 1470), ('듣', 1454), ('지구', 1432), ('中', 1425), ('만들', 1411), ('李', 1394), ('화하', 1390), ('상소', 1382), ('道', 1372), ('수', 1358),

- 아래는 온/오프라인별 주요 단어 추출

In [45]:
hobbysonoff=[]
fread=open("./Leisures/Text/비대면_취미.txt", 'r',encoding='UTF8')
templist=fread.read()
tempresult=mecab_nouns(templist)
text=nltk.Text(tempresult,name='비대면_취미')
hobbysonoff.append(text.vocab().most_common(200))

In [46]:
fread=open("./Leisures/Text/온라인_취미.txt", 'r',encoding='UTF8')
templist=fread.read()
print(len(templist))
tempresult=mecab_nouns(templist)
text=nltk.Text(tempresult,name='온라인_취미')
hobbysonoff.append(text.vocab().most_common(200))

1632972


In [47]:
fread=open("./Leisures/Text/오프라인_취미.txt", 'r',encoding='UTF8')
templist=fread.read()
tempresult=mecab_nouns(templist)
text=nltk.Text(tempresult,name='오프라인_취미')
hobbysonoff.append(text.vocab().most_common(200))

In [48]:
fread=open("./Leisures/Text/야외_취미.txt", 'r',encoding='UTF8')
templist=fread.read()
tempresult=mecab_nouns(templist)
text=nltk.Text(tempresult,name='야외_취미')
hobbysonoff.append(text.vocab().most_common(200))

In [76]:
hobbylist=[]
for row in hobbysonoff:
    hobby=[]
    for obj in row:
        hobby.append(obj[0])
        print(type(obj[1]))
    hobbylist.append(hobby)

<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class

In [41]:
if '마음' in hobbylist:
    print("hello")

In [50]:
onlinelist=hobbylist[0]+hobbylist[1]
offlinelist=hobbylist[2]+hobbylist[3]
online=[]
offline=[]
for obj in onlinelist:
    if obj not in offlinelist:
        online.append(obj)
for obj in offlinelist:
    if obj not in onlinelist:
        offline.append(obj)

In [51]:
online

['비대',
 '피아노',
 '기업',
 '시대',
 '학교',
 '대면',
 '사회',
 '택트',
 '언',
 '학생',
 '단체',
 '상황',
 '전화',
 '화상',
 '한국',
 '학습',
 '문의',
 '방식',
 '보컬',
 '기관',
 '도서관',
 '출강',
 '변화',
 '줌',
 '진주',
 '사업',
 '봉사',
 '원예',
 '청소년',
 '관리',
 '감사',
 '인해',
 '정원',
 '예술',
 '대상',
 '영어',
 '목공',
 '지갑',
 '룻',
 '콘텐츠',
 '모집',
 '실',
 '소통',
 '안내',
 '준비물',
 '마스터',
 '비스킷',
 '아이패드',
 '배워',
 '츠',
 '마크라메',
 '색연필',
 '프리',
 '패스',
 '클',
 '사회',
 '편하',
 '래스',
 '마이비',
 '평생',
 '스킷',
 '요리',
 '하비',
 '프랑스',
 '장점',
 '파스텔',
 '비누',
 '간단',
 '일러스트',
 '비대',
 '부담',
 '강',
 '만든',
 '오일',
 '핸드',
 '패키지',
 '글씨']

In [44]:
offline

['야외',
 '촬영',
 '캠핑',
 '스케치',
 '화실',
 '등산',
 '발레',
 '웨딩',
 '시장',
 '자연',
 '커피',
 '자전거',
 '풍경',
 '분위기',
 '나가',
 '이야기',
 '행복',
 '나무',
 '다니',
 '맛있',
 '부케',
 '가족',
 '가을',
 '자리',
 '미니',
 '공원',
 '꽃집',
 '갔',
 '느끼',
 '산',
 '엄마',
 '주문',
 '산책',
 '드론',
 '스포츠',
 '넓',
 '보여',
 '합니다',
 '밖',
 '장소',
 '스트레스',
 '혼자',
 '시원',
 '이야기',
 '매장',
 '읽',
 '정보',
 '구입',
 '샵',
 '경험',
 '관련',
 '와인',
 '상품',
 '인터넷',
 '창업',
 '청년',
 '앱',
 '플',
 '파티',
 '드',
 '쇼핑',
 '달',
 '갖',
 '발레',
 '홈페이지',
 '이유',
 '카드',
 '공유',
 '목표']

In [142]:
fread=open("./Leisures/Text/맨손_체조_참여.txt", 'r',encoding='UTF8')
templist=fread.read()
tempresult=mecab_nouns(templist)
text=nltk.Text(tempresult,name='외식')
text.vocab().most_common(250)

[('운동', 5322),
 ('시간', 2355),
 ('체조', 2278),
 ('어르신', 2256),
 ('활동', 2108),
 ('건강', 1896),
 ('센터', 1825),
 ('사람', 1616),
 ('프로그램', 1523),
 ('참여', 1481),
 ('보호', 1319),
 ('생활', 1239),
 ('생각', 1169),
 ('몸', 1165),
 ('노인', 1087),
 ('시작', 1048),
 ('신체', 1044),
 ('주간', 1043),
 ('진행', 1009),
 ('체육', 978),
 ('손', 947),
 ('만들', 940),
 ('교육', 919),
 ('정도', 918),
 ('먹', 874),
 ('방법', 859),
 ('대한', 855),
 ('길', 832),
 ('다양', 820),
 ('학교', 806),
 ('이용', 801),
 ('준비', 791),
 ('필요', 789),
 ('걷', 786),
 ('아이', 775),
 ('마음', 771),
 ('이상', 765),
 ('근육', 756),
 ('자신', 735),
 ('따라', 725),
 ('수업', 716),
 ('효과', 708),
 ('학생', 686),
 ('맨손', 671),
 ('동작', 671),
 ('하루', 635),
 ('곳', 635),
 ('찾', 628),
 ('집', 626),
 ('스트레칭', 621),
 ('스포츠', 618),
 ('사회', 614),
 ('모습', 612),
 ('잡', 609),
 ('관리', 606),
 ('사용', 604),
 ('이야기', 600),
 ('치료', 597),
 ('아침', 588),
 ('중요', 586),
 ('책', 582),
 ('기능', 579),
 ('놀이', 567),
 ('도움', 554),
 ('학습', 553),
 ('체력', 545),
 ('예방', 541),
 ('과정', 540),
 ('읽', 534),
 ('활용', 531),
 ('운영